In [ ]:
from model import GRP
from libriichi.dataset import Grp
from train_grp import GrpFileDatasetsIter
import torch
import glob
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence

def collate(batch):
    inputs = []
    lengths = []
    rank_by_players = []
    for inputs_seq, rank_by_player in batch:
        inputs.append(inputs_seq)
        lengths.append(len(inputs_seq))
        rank_by_players.append(rank_by_player)

    lengths = torch.tensor(lengths)
    rank_by_players = torch.tensor(rank_by_players, dtype=torch.int64)

    padded = pad_sequence(inputs, batch_first=True)
    packed_inputs = pack_padded_sequence(padded, lengths, batch_first=True, enforce_sorted=False)

    return packed_inputs, rank_by_players
train_file_list = glob.glob("dataset/train/**/*.json.gz", recursive=True)
train_file_data = GrpFileDatasetsIter(
    file_list = train_file_list,
    file_batch_size = 50,
    cycle = True,
)
train_data_loader = iter(DataLoader(
    dataset = train_file_data,
    batch_size = 1,
    drop_last = True,
    num_workers = 0,
    collate_fn = collate,
))
grp = GRP()

grp.eval()
for inputs, rank_by_player in train_data_loader:
    logits = grp.forward_packed(inputs)
    label = grp.get_label(rank_by_player)
    print(logits, label)
    break




RuntimeError: Need to provide pin_memory allocator to use pin memory.